# Introduction

1. I changed the keras code so that I could use it to update the actor model with a custom gradient
2. Not sure this is the best environment to train on -- the rewards 

In [2]:
import gym
import time
import numpy as np
import matplotlib.pyplot as plt
from agent import Agent
from collections import Counter
%matplotlib inline

#Environment
seed = 14
env = gym.make('MountainCar-v0')
env.seed(seed)  # for comparison
num_states = env.observation_space.shape[0]
num_actions = env.action_space.n

#Agent
gamma, lr, tau = 0.99, 0.0001, 0.01
agent = Agent(num_states, num_actions, lr, gamma, seed_num = seed)
agent.memory_size = 10**5
agent.batchsize = 256
learning_start = 25*agent.batchsize
agent.tau = tau


#Train
EPISODES = 500
global_step = 0
scores = []
for e in range(1,EPISODES+1):
    state = env.reset()
    reward_sum = 0
    done = False
    steps = 0
    xmin = state[0]
    xmax = state[0]
    t1 = time.time()
    actions = []
    while not done:
        #env.render()
        state = np.reshape(state, [1, num_states])  #reshape for keras
        action_onehot = agent.act(state)
        action_scalar = np.dot(action_onehot,range(num_actions))
        actions.append(action_scalar)
        #print action_scalar
        next_state, reward, done, _ = env.step(action_scalar)
        reward_sum += reward
        agent.remember(state[0], action_onehot, reward, next_state, done)
        state = next_state
        
        if len(agent.memory) > learning_start and global_step % 50 == 0:
            agent.train_models()
        
        steps += 1
        global_step += 1
        x = state[0]
        xmax = max(xmax, x)
        xmin = min(xmin, x)
    
    #Learn & print results
    scores.append(reward_sum)
    t2 = time.time()
    if e % 10 == 0:
        t2 = time.time()
        xmax = np.round(xmax,3)
        xmin = np.round(xmin, 3)
        print '(ep, score, xmin, xmax, T (mins)) = ' + str((e,reward_sum, xmin, xmax, (t2-t1)/60.0))
        print 'actions counts = ' + str(Counter(actions).values()) + '\n'
        
#agent.save_target_weights()
plt.plot(scores)
#np.savetxt('stats/scores_inverted_pendulum.txt',scores)

WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.
(ep, score, xmin, xmax, T (mins)) = (10, -200.0, -0.618, -0.42, 0.0013969659805297852)
actions counts = [34, 122, 44]

(ep, score, xmin, xmax, T (mins)) = (20, -200.0, -0.628, -0.408, 0.0014989137649536132)
actions counts = [31, 133, 36]

(ep, score, xmin, xmax, T (mins)) = (30, -200.0, -0.685, -0.331, 0.0015303492546081543)
actions counts = [33, 112, 55]

(ep, score, xmin, xmax, T (mins)) = (40, -200.0, -0.567, -0.477, 0.008382868766784669)
actions counts = [44, 124, 32]

(ep, score, xmin, xmax, T (mins)) = (50, -200.0, -0.707, -0.352, 0.008450949192047119)
actions counts = [37, 107, 56]

(ep, score, xmin, xmax, T (mins)) = (60, -200.0, -0.582, -0.469, 0.008336047331492106)
actions counts = [40, 120, 40]

(ep, score, xmin, xmax, T (mins)) = (70, -200.0, -0.577, -0.496, 0.008740917841593424)
actions counts = [47, 128, 25]

(ep, score, xmin, xmax, T (mins)) = (80, -200.0, -0.699, -0.429, 0.

KeyboardInterrupt: 

### Watch a smart agent

In [3]:
len(agent.memory)

10001

In [4]:
env.reset()
done = False
while not done:
    #env.render()
    state = np.reshape(state, [1, num_states])  #reshape for keras
    action_onehot = agent.act(state)
    action_scalar = np.dot(action_onehot,range(num_actions))
    next_state, reward, done, _ = env.step(action_scalar)
    reward_sum += reward
    state = next_state
    print action_scalar
env.close()

NameError: name 'env' is not defined